In [ ]:
import sys, os, time
import numpy as np
sys.path.append('dti_reference_implementation')

from deep_tissue_imaging.elementos.lasers import fuente_microscopia_1 as laser, campo_tem00
from deep_tissue_imaging.elementos.tejidos import cerebro_emb_pez_cebra as tejido
from deep_tissue_imaging.elementos import domain as Domain
from deep_tissue_imaging.elementos.plotting import plot_field_intensity
from benchmark.phase_mask_manager import PhaseMaskManager
from pynq import Overlay, allocate


In [ ]:
# Domain parameters
Lz = np.float32(361e-6)
Nz = 361
dz = np.float32(Lz / Nz)

Lx, Ly = np.float32(45e-6), np.float32(45e-6)
Nx, Ny = 256, 256
dx = np.float32(Lx / Nx)
dy = np.float32(Ly / Ny)

x = np.linspace(-Lx/2, Lx/2, Nx, dtype=np.float32)
y = np.linspace(-Ly/2, Ly/2, Ny, dtype=np.float32)
X, Y = np.meshgrid(x, y)

k0 = np.float32(2*np.pi / laser.wavelength)
k = np.float32(k0 * tejido.n_0)
sigma_phi = np.float32(k * tejido.Dn * tejido.l_s)
sigma_x = np.float32(5e-6)

domain = Domain.Domain(X, Y, Nx, Ny, Nz, dx, dy, dz, np.float32(1e-12), k0, k, sigma_phi, sigma_x)


In [ ]:
# Create TEM00 initial beam profile
phi = campo_tem00(X, Y, laser.w0, laser.I_peak)

# Phase mask manager using stored masks
mask_manager = PhaseMaskManager(save_dir='./dti_reference_implementation/phase_masks')
mask_manager.initialize_masks(phi.shape, X, Y, domain.sigma_phi, domain.sigma_x)


In [ ]:
# Load overlay and define helper to run accelerator
overlay = Overlay('dti_bpm.bit')  # assume overlay file exists
bpm_ip = overlay.bpm_accel_0  # accelerator IP block

# Helper function to run propagation for a number of steps
def run_propagation(field, steps):
    buf_in = allocate(shape=field.shape, dtype=np.complex64)
    buf_out = allocate(shape=field.shape, dtype=np.complex64)
    buf_in[:] = field
    bpm_ip.write(bpm_ip.register_map.input_r.address, buf_in.physical_address)
    bpm_ip.write(bpm_ip.register_map.output_r.address, buf_out.physical_address)
    bpm_ip.write(bpm_ip.register_map.nsteps.address, np.uint32(steps))
    bpm_ip.register_map.CTRL.AP_START = 1
    while bpm_ip.register_map.CTRL.AP_DONE == 0:
        pass
    result = np.array(buf_out)
    buf_in.freebuffer(); buf_out.freebuffer()
    return result

# Execute 3 segments of 120 steps each with phase masks
t0 = time.time()
for idx in range(1, 4):
    phi = run_propagation(phi, 120)
    phi = mask_manager.apply_mask(phi, idx)
t_exec = time.time() - t0
print(f'Execution time: {t_exec:.6f} seconds')


In [ ]:
# Save and plot final field
np.save('final_field.npy', phi)
plot_field_intensity(phi, X, Y)
